In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import os
sys.path.append(os.path.abspath('/Users/guhansundar/Documents/GuData/ObjectiveSubjectiveHealth/'))
from scripts.parse_surveys import _resolve_survey_app_id_from_db, _prepare_wide_by_app_from_db, get_survey_data_for_subject, list_available_surveys


In [2]:
DATABASE_PATH = '/Users/guhansundar/Documents/GuData/ObjectiveSubjectiveHealth/data/database.db'
conn = sqlite3.connect(DATABASE_PATH)


In [3]:
events_df = pd.read_sql_query("SELECT * FROM events", conn)

In [ ]:
events_df.columns

In [ ]:
DATABASE_PATH = '/Users/guhansundar/Documents/GuData/ObjectiveSubjectiveHealth/data/database.db'
conn = sqlite3.connect(DATABASE_PATH)


subject_id = 'demo'

# Query the data
query = f"""
SELECT *
FROM events
WHERE app_type = 'survey'
and subject_id = '{subject_id}'
"""
df = pd.read_sql_query(query, conn)

In [ ]:
df

In [ ]:
print(list_available_surveys(conn, subject_id='demo'))

In [4]:
df = get_survey_data_for_subject(subject_id='demo', survey='daily_core', conn=conn)


In [5]:
df

,energy_1to5__value,happiness_1to5__value,productive_1to5__value,social_fulfillment_1to5__value,stress_1to5__value,ts_utc
session_id,,,,,,
2025-08-16 08:41:20.483000+00:00,5.0,4.0,5.0,2.0,3.0,2025-08-16 21:12:58.641000+00:00


In [4]:
subject_id = 'demo'
survey= 'daily_core'
table= "events"
include_session_as_index = True

"""
Return a dataframe for a given subject and survey (app_id) where:
    - rows = sessions (timestamp)
    - columns = metrics from that survey's payload, e.g., "<item_id>__value", "<item_id>__rt"
Works for all surveys present in the DB.

Args:
    subject_id: subject identifier to filter on.
    survey: survey identifier (prefer exact app_id, e.g., 'demo_all_inputs_v1').
            Substrings are allowed and resolved if unambiguous.
    conn: sqlite3 connection (e.g., sqlite3.connect(DB_PATH)).
    table: DB table name (default 'events').
    include_session_as_index: if True, return df indexed by timestamp.

Returns:
    pd.DataFrame with rows as timestamp and columns as survey metric columns.
"""
app_id = _resolve_survey_app_id_from_db(conn, survey, subject_id=subject_id, table=table)

# Build a wide table for just this subject & this survey
wide = _prepare_wide_by_app_from_db(conn, subject_id=subject_id, app_id=app_id, table=table)
if wide.empty:
    raise ValueError(f"No rows found for subject_id='{subject_id}' and survey(app_id)='{app_id}'.")

# Keep only metric columns (drop identifiers)
# id_cols = {"subject_id", "session_id", "app_id", "app_type"}
# metric_cols = [c for c in wide.columns if c not in id_cols]

# out = wide[["ts_utc"] + metric_cols].copy()

# # Convert 'ts_utc' to datetime and set as index
# out['ts_utc'] = pd.to_datetime(out['ts_utc'], utc=True)

# # One row per timestamp (if duplicates, take last non-null per column)
# out = (
#     out.groupby("ts_utc", dropna=False)
#     .agg(_last_nonnull)
#     .reset_index()
#     .sort_values("ts_utc")
# )

# if include_session_as_index:
#     out = out.set_index("ts_utc")

# # Consistent column order
# out = out.reindex(sorted(out.columns), axis=1)


In [ ]:
print("Current PYTHONPATH:", sys.path)

In [ ]:
print("Current PYTHONPATH:", sys.path)

In [5]:
wide

,subject_id,session_id,app_id,app_type,ts_utc,happiness_1to5__value,energy_1to5__value,stress_1to5__value,productive_1to5__value,social_fulfillment_1to5__value
0,demo,2025-08-16 08:41:20.483000+00:00,daily_core,survey,2025-08-16 21:12:58.641000+00:00,4.0,5.0,3.0,5.0,2.0
